# A10_5A_2020_SoCSE_KLETECH
## COVID19 Fake News Detection in English(5DMACP06)

### Problem Statement
Detection of COVID-19 related fake news in English.
Given a social media post, the objective of the shared task is to classify it into either fake or real news.
For example, the following two posts belong to fake and real categories, respectively. 
- If you take Crocin thrice a day you are safe.--->     Fake
- Wearing mask can protect you from the virus. ---> 	Real

| Name | USN | Roll No. |
| --- | --- | --- |
| Anup Hadalageri| 01fe18bcs041| 141 |
| Ayazahemad Shaikh| 01fe18bcs058| 158 |
| Ankita Patil| 01fe18bcs039| 139 |
| Abhinav Kumar| 01fe18bcs008| 108 |

### Importing all the required packages.

In [2]:
#Mount the Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#import the required libraries
import pandas as pd
import numpy as np
import itertools
import nltk, os, re, string
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from sklearn import metrics
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from nltk.corpus import wordnet
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Importing Dataset

In [4]:
#Load the Train, Validation and Test dataset
train_data = pd.read_csv('/content/drive/My Drive/DMA/Final/Constraint_English_Train - Sheet1.tsv', quoting = 3, delimiter='\t')
val_data = pd.read_csv('/content/drive/My Drive/DMA/Final/Constraint_English_Val - Sheet1.tsv', quoting = 3, delimiter='\t')
test_data = pd.read_csv('/content/drive/My Drive/DMA//Final/Constraint_English_Test - Sheet1 .tsv', quoting = 3, delimiter='\t')

### Dataset Description 

In [5]:
#printing the train dataset
print(train_data)

        id                                              tweet label
0        1  The hapening Shouldn't  CDC currently reports ...  real
1        2  States reported 1121 deaths a small rise from ...  real
2        3  Politically Correct Woman (Almost) Uses Pandem...  fake
3        4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4        5  Populous states can generate large case counts...  real
...    ...                                                ...   ...
6381  6416  A tiger tested positive for COVID-19 please st...  fake
6382  6417  ???Autopsies prove that COVID-19 is??� a blood...  fake
6383  6418  _A post claims a COVID-19 vaccine has already ...  fake
6384  6419  Aamir Khan Donate 250 Cr. In PM Relief Cares Fund  fake
6385  6420  It has been 93 days since the last case of COV...  real

[6386 rows x 3 columns]


In [6]:
#printing the Validation dataset
print(val_data)

        id                                              tweet label
0        1  The hapening Shouldn't  CDC currently reports ...  real
1        2  States reported 1121 deaths a small rise from ...  real
2        3  Politically Correct Woman (Almost) Uses Pandem...  fake
3        4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4        5  Populous states can generate large case counts...  real
...    ...                                                ...   ...
6371  6406  The cloth face coverings recommended to slow s...  real
6372  6407  Video shows Merkel celebrating the new COVID-1...  fake
6373  6408  @KagAshley @daveman19802 @charliekirk11 Ashley...  fake
6374  6409  _American citizens have placed a nude statue o...  fake
6375  6410  @RickLeventhal @RepAndyBiggsAZ If covid is so ...  fake

[6376 rows x 3 columns]


In [7]:
#printing the Number of rows, Number of attributes, Data type of each attribute and the top 5 tuples from the train dataset
print("No of rows in the training set:", len(train_data))
print("--------------------------")
print("No of attributes:",len(train_data.columns))
print("--------------------------")
print(train_data.dtypes)
print("--------------------------")
print(train_data.head())
print("--------------------------")

No of rows in the training set: 6386
--------------------------
No of attributes: 3
--------------------------
id        int64
tweet    object
label    object
dtype: object
--------------------------
   id                                              tweet label
0   1  The hapening Shouldn't  CDC currently reports ...  real
1   2  States reported 1121 deaths a small rise from ...  real
2   3  Politically Correct Woman (Almost) Uses Pandem...  fake
3   4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4   5  Populous states can generate large case counts...  real
--------------------------


In [8]:
#printing the Number of rows, Number of attributes, Data type of each attribute and the top 5 tuples from the Validation dataset
print("No of rows in the Validation set:", len(val_data))
print("--------------------------")
print("No of attributes:",len(val_data.columns))
print("--------------------------")
print(val_data.dtypes)
print("--------------------------")
print(val_data.head())
print("--------------------------")

No of rows in the Validation set: 6376
--------------------------
No of attributes: 3
--------------------------
id        int64
tweet    object
label    object
dtype: object
--------------------------
   id                                              tweet label
0   1  The hapening Shouldn't  CDC currently reports ...  real
1   2  States reported 1121 deaths a small rise from ...  real
2   3  Politically Correct Woman (Almost) Uses Pandem...  fake
3   4  #IndiaFightsCorona: We have 1524 #COVID testin...  real
4   5  Populous states can generate large case counts...  real
--------------------------


### Checking for null values


In [9]:
print('Part of missing values for every column')
print(train_data.isnull().sum()/ len(train_data))

Part of missing values for every column
id       0.0
tweet    0.0
label    0.0
dtype: float64


### Checking for duplicate ids

In [10]:
unique_users=train_data.id.nunique()
print("We have",unique_users,"number of unique users in the training set")

We have 6386 number of unique users in the training set


## Pre-processing 

### List of techniques used: 


- Tokenization
- Lemmatization
- Lowercase
- Url Remover
- Spell Correction
- Special Characters Removal
- Changing the lables from Categorical values to binary

### Changing the lables from Categorical values to binary  

#### The values of the lables are changed from real and fake to 1's and 0's respectively to decrease the computational time.

### Here the data is pre-processed and Output is show below


## Word Embedding - word2vec


In [11]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
import numpy as np

### one hot representaion
#### In the one hot representation the words in the data are mapped to numbers as shown below.
#### Here padding is done to ensure that all the sentences in the matrix are of same length.


In [12]:
def conv2array(text,label):
    X_train=np.array(text)
    y_train=np.array(label)
    return X_train,y_train

In [13]:
from sklearn.preprocessing import LabelEncoder
def changingLabel(data):
    le = LabelEncoder()
    y = data.iloc[:,-1].values
    y= le.fit_transform(y)
    return y

In [14]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [16]:
import gensim

In [17]:
EMBEDDING_DIM = 100
maxlen = 30
sent_length = 30

In [18]:
def preprocessing(data):
    X = []
    stop_words = set(nltk.corpus.stopwords.words("english"))
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    for par in data["tweet"].values:
        tmp = []
        sentences = nltk.sent_tokenize(par)
        for sent in sentences:
            sent = sent.lower()
            tokens = tokenizer.tokenize(sent)
            filtered_words = [w.strip() for w in tokens if w not in stop_words and len(w) > 1]
            tmp.extend(filtered_words)
        X.append(tmp)
    
    return X

In [19]:
def tokenizeTheWords(X):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X)

    X = tokenizer.texts_to_sequences(X)
    word_index = tokenizer.word_index
    return X,word_index

In [20]:
"""for word, num in word_index.items():
    print(f"{word} -> {num}")
    if num == 10:
        break  """


'for word, num in word_index.items():\n    print(f"{word} -> {num}")\n    if num == 10:\n        break  '

In [21]:
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = np.zeros((6387, EMBEDDING_DIM))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [22]:
# Main Function
X= preprocessing(train_data)
w2v_model = gensim.models.Word2Vec(sentences=X, size=EMBEDDING_DIM, window=5, min_count=1)
X,word_index = tokenizeTheWords(X)
#embedding_vectors = get_weight_matrix(w2v_model, word_index)
X_train = pad_sequences(X, maxlen=maxlen)
y_train = changingLabel(train_data)
 
# val

val = preprocessing(val_data)
val,word_index_val = tokenizeTheWords(val)
X_val = pad_sequences(val, maxlen=maxlen)
y_val = changingLabel(val_data)

In [23]:
X_train.shape

(6386, 30)

In [24]:
#embedding_vectors[2]

CNN

In [25]:
from keras.layers import Dense
from keras.layers import Flatten, GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.layers import Dropout

embedding_vector_features=100
model=Sequential()
model.add(Embedding(150000,embedding_vector_features,input_length=sent_length))
model.add(Conv1D(256, 3, padding='same', activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 30, 100)           15000000  
_________________________________________________________________
conv1d (Conv1D)              (None, 30, 256)           77056     
_________________________________________________________________
global_max_pooling1d (Global (None, 256)               0         
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               64250     
_________________________________________________________________
dropout_1 (Dropout)          (None, 250)               0

In [27]:
model.fit(X_train,y_train,epochs=2,batch_size=64)

Epoch 1/2
100/100 [==============================] - 18s 169ms/step - loss: 0.5968 - accuracy: 0.6478
Epoch 2/2
100/100 [==============================] - 17s 168ms/step - loss: 0.1468 - accuracy: 0.9433


In [28]:
len(model.layers)

8

In [29]:
scores = model.evaluate(X_val, y_val, verbose=50)

In [30]:
y_pred=model.predict_classes(X_val)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val,y_pred)

93.13739021329987

In [34]:
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 93.14%


In [36]:
print(y_pred)

[[1]
 [1]
 [0]
 ...
 [1]
 [0]
 [0]]


In [40]:
for i in y_pred:
  if (i==1):
    print("Real")
  else:
    print("Fake")

Streaming output truncated to the last 5000 lines.
Fake
Real
Real
Real
Real
Fake
Real
Fake
Real
Real
Fake
Real
Fake
Real
Fake
Real
Real
Real
Real
Real
Fake
Fake
Fake
Fake
Real
Fake
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Fake
Real
Real
Fake
Real
Fake
Fake
Real
Fake
Real
Real
Fake
Real
Real
Fake
Fake
Fake
Fake
Real
Fake
Real
Real
Real
Fake
Fake
Real
Fake
Real
Real
Real
Real
Fake
Fake
Fake
Fake
Fake
Real
Real
Fake
Real
Real
Fake
Real
Real
Fake
Real
Fake
Real
Real
Real
Real
Fake
Fake
Real
Real
Real
Real
Real
Fake
Fake
Fake
Real
Fake
Real
Fake
Fake
Fake
Fake
Fake
Real
Real
Real
Fake
Fake
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Fake
Fake
Fake
Real
Fake
Fake
Fake
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Fake
Fake
Fake
Fake
Fake
Real
Real
Real
Fake
Real
Fake
Fake
Fake
Real
Real
Fake
Real
Fake
Fake
Real
Real
Real
Real
Fake
Fake
Fake
Real
Fake
Real
Real
Real
Real
Real
Fake
Fake
Fake
Real
Fake
Real
Real
Real
Fake
Real
Real
Fake
Real
Real
Fake
Fake